In [37]:
df = pd.read_csv("data/cleaned_global_youtube_statistics.csv")
df_country = pd.read_csv("data/cleaned_country_data.csv")
df.columns

Index(['rank', 'Youtuber', 'subscribers', 'video views', 'category', 'uploads',
       'Country', 'video_views_rank', 'country_rank',
       'highest_monthly_earnings', 'created_year',
       'Gross tertiary education enrollment (%)', 'Unemployment rate'],
      dtype='object')

In [42]:
year_counts = df['created_year'].value_counts()

year_counts_dict = year_counts.to_dict()

In [44]:
from dash import Dash, dcc, Output, Input, html  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px
import pandas as pd                        # pip install pandas
import webbrowser
from threading import Timer

# Load the dataset
file_path = 'data/cleaned_global_youtube_statistics.csv'
df = pd.read_csv(file_path)

# Create the dictionary for created_year value counts
created_year_counts = df['created_year'].value_counts().to_dict()

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])

# Create individual dropdowns
heatmap_dropdown = dcc.Dropdown(
    options=[{'label': col, 'value': col} for col in df.columns],
    value='subscribers',  # initial value displayed when page first loads
    clearable=False,
    id='heatmap-dropdown'
)

histogram_dropdown = dcc.Dropdown(
    options=[{'label': col, 'value': col} for col in df.columns],
    value='created_year',  # Set initial value to 'created_year'
    clearable=False,
    id='histogram-dropdown'
)

scatterplot_dropdown = dcc.Dropdown(
    options=[{'label': col, 'value': col} for col in df.columns],
    value='subscribers',
    clearable=False,
    id='scatterplot-dropdown'
)

wordcloud_dropdown = dcc.Dropdown(
    options=[{'label': col, 'value': col} for col in df.columns],
    value='subscribers',
    clearable=False,
    id='wordcloud-dropdown'
)

barchart_dropdown = dcc.Dropdown(
    options=[{'label': col, 'value': col} for col in df.columns],
    value='subscribers',
    clearable=False,
    id='barchart-dropdown'
)

heatmap = dcc.Graph(id='heatmap')
histogram = dcc.Graph(id='histogram')
scatterplot = dcc.Graph(id='scatterplot')
wordcloud = dcc.Graph(id='wordcloud')
barchart = dcc.Graph(id='barchart')

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([heatmap_dropdown, heatmap], width=6),  # Heatmap on the left
        dbc.Col([
            histogram_dropdown, histogram,  # Histogram on top right
            scatterplot_dropdown, scatterplot  # Scatterplot below histogram
        ], width=6)
    ]),
    dbc.Row([
        dbc.Col([wordcloud_dropdown, wordcloud], width=6),  # Wordcloud on the left
        dbc.Col([barchart_dropdown, barchart], width=6)  # Bar chart on the right
    ])
], fluid=True)

# Callback allows components to interact
@app.callback(
    Output('heatmap', 'figure'),
    Input('heatmap-dropdown', 'value')
)
def update_heatmap(column_name):  # Function arguments come from the component property of the Input

    print(column_name)
    print(type(column_name))

    heatmap_fig = px.choropleth(
        data_frame=df,
        locations='Country',  # Assuming 'Country' contains ISO country codes
        locationmode="country names",
        height=700,
        color=column_name,
        title=f'Heatmap of {column_name} by Country'
    )

    return heatmap_fig

@app.callback(
    Output('histogram', 'figure'),
    Input('histogram-dropdown', 'value')
)
def update_histogram(column_name):
    if column_name == 'created_year':
        x_values = list(created_year_counts.keys())
        y_values = list(created_year_counts.values())
        histogram_fig = px.bar(x=x_values, y=y_values, labels={'x': 'Year', 'y': 'Count'}, title=f'Histogram of Created Year')
    else:
        histogram_fig = px.histogram(df, x=column_name, title=f'Histogram of {column_name}')
    return histogram_fig

@app.callback(
    Output('scatterplot', 'figure'),
    Input('scatterplot-dropdown', 'value')
)
def update_scatterplot(column_name):

    scatterplot_fig = px.scatter(df, x='created_year', y=column_name, title=f'Scatterplot of {column_name} over Years')
    return scatterplot_fig

@app.callback(
    Output('wordcloud', 'figure'),
    Input('wordcloud-dropdown', 'value')
)
def update_wordcloud(column_name):

    wordcloud_fig = px.scatter(df, x='created_year', y=column_name, title=f'Wordcloud Placeholder for {column_name}')
    return wordcloud_fig

@app.callback(
    Output('barchart', 'figure'),
    Input('barchart-dropdown', 'value')
)
def update_barchart(column_name):

    barchart_fig = px.bar(df, x='Youtuber', y=column_name, title=f'Bar Chart of {column_name} by Youtuber')
    return barchart_fig

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8054/")

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True, port=8054)


subscribers
<class 'str'>
subscribers
<class 'str'>
subscribers
<class 'str'>
subscribers
<class 'str'>
subscribers
<class 'str'>
